In [1]:
import Pkg; 

cd(joinpath(@__DIR__, "../../"))
Pkg.activate("Project.toml")

using MorphoMol
using PyCall
using JLD2
using LinearAlgebra
using NearestNeighbors
using Rotations

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`


In [2]:
using PyCall

function get_interface_diagram(points, n_atoms_per_mol)
    py"""
    import oineus as oin
    import numpy as np
    import torch
    import diode

    def get_interface_diagram(points, n_atoms_per_mol):
        points = np.asarray(points)
        print("PYTHON:")
        print("x: ", min([p[0] for p in points]), " ", max([p[0] for p in points]))
        print("y: ", min([p[1] for p in points]), " ", max([p[1] for p in points]))
        print("z: ", min([p[2] for p in points]), " ", max([p[2] for p in points]))

        simplices = diode.fill_alpha_shapes(points)
        # #print(simplices)
        # for sim in simplices:
        #     if len(sim[0]) == 1:
        #          print(sim)
        
        fil = oin.Filtration_double([oin.Simplex_double(s[0], s[1]) for s in simplices])

        def is_multi(sigma):
            has_a = has_b = False
            for v in sigma.vertices:
                if v < n_atoms_per_mol:
                    has_a = True
                else:
                    has_b = True
            return has_a and has_b

        fil = fil.subfiltration(is_multi)
        # second argument: True for cohomology, False for homology (incorrect for subfiltration)
        dcmp = oin.Decomposition(fil, True)
        params = oin.ReductionParams()
        params.clearing_opt = False
        dcmp.reduce(params)
        #odgm = dcmp.diagram(fil, include_inf_points=False)
        dgm = dcmp.diagram(fil, include_inf_points=False).in_dimension(2, as_numpy=False)
        print("OINEUS")
        print(fil.cells()[3632])
        print(fil.cells()[5180])
        print(dgm)
        #return dgm
    """
    py"get_interface_diagram"(points, n_atoms_per_mol)
end

get_interface_diagram_as_numpy_false (generic function with 1 method)

In [ ]:
@load "assets/output/persistence/exploding.jld2" input output
display(interactive_simulation_analysis(merge!(input, Dict("mol_type" => "6r7m")), output))

points = MorphoMol.Utilities.get_matrix_realization(output["states"][end], input["template_centers"])
points = [e for e in eachcol(hcat(points...))];
println("JULIA:")
println("x: ", minimum([p[1] for p in points]), " ", maximum([p[1] for p in points]))
println("y: ", minimum([p[2] for p in points]), " ", maximum([p[2] for p in points]))
println("z: ", minimum([p[3] for p in points]), " ", maximum([p[3] for p in points]))

dgms = get_interface_diagram(points, 1206)
dgms